# Milestone 2

Brandon Sams

19Apr2020

## Perform at least 5 data transformation and/or cleansing steps to your flat file data. For example:
	• Replace Headers
	• Format data into a more readable format
	• Identify outliers and bad data
	• Find duplicates
	• Fix casing or inconsistent values
	• Conduct Fuzzy Matching

In [31]:
import pandas as pd

# Import the csv from a flat file
pdx_crime = pd.read_csv('Open_Data_Sheet_data.csv')
pdx_crime.shape

(202552, 17)

In [32]:
# Replace Headers
headers = ['address','case_num','crime_against','neighborhood','num_records','occur_date','occur_month_year','occur_time','offense_category','offense_count','offense_type','lat','long','x','y','report_date','report_month_year']

pdx_crime.columns = headers

In [33]:
# Format data into a more readable format

# Make dates into datetime objects
date_attributes = ['occur_date','occur_month_year','report_date','report_month_year']
for att in date_attributes:
    pdx_crime[att] = pd.to_datetime(pdx_crime[att], infer_datetime_format=True)

In [34]:
# Identify outliers and bad data
def detect_iqr(ser):
    Q1 = ser.quantile(0.25)
    Q3 = ser.quantile(0.75)
    IQR = Q3 - Q1
    return((ser < (Q1 - 1.5 * IQR)) | (ser > (Q3 + 1.5 * IQR)))

print(f'There are {pdx_crime[detect_iqr(pdx_crime["lat"]) == True].shape[0]} rows where the latitude is considered an outlier.')
print(f'There are {pdx_crime[detect_iqr(pdx_crime["long"]) == True].shape[0]} rows where the longitude is considered an outlier.')

There are 8290 rows where the latitude is considered an outlier.
There are 68 rows where the longitude is considered an outlier.


In [35]:
# Find duplicates

# column with a single value
print(f'num_records has {pdx_crime.num_records.value_counts().shape[0]} distinct value')
pdx_crime.drop(columns = ['num_records'], inplace = True)

# count duplicate rows
print(f'There are {pdx_crime[pdx_crime.duplicated() == True].shape[0]} duplicated rows')
pdx_crime.drop_duplicates(keep=False,inplace=True)

num_records has 1 distinct value
There are 2354 duplicated rows


In [36]:
# Fix casing or inconsistent values
string_attributes = ['address','crime_against','neighborhood','offense_category','offense_type']
for att in string_attributes:
    pdx_crime[att] = pdx_crime[att].str.upper()

In [37]:
df = pdx_crime

In [38]:
df

,address,case_num,crime_against,neighborhood,occur_date,occur_month_year,occur_time,offense_category,offense_count,offense_type,lat,long,x,y,report_date,report_month_year
0,3600 BLOCK OF SE KNAPP ST,17-902332,PROPERTY,EASTMORELAND,2017-02-20,2017-02-01,0,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.470545,-122.625298,7656952.0,664817.0,2017-02-20,2017-02-01
1,3600 BLOCK OF SE LAMBERT ST,17-902346,PROPERTY,EASTMORELAND,2017-02-20,2017-02-01,30,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.467028,-122.625272,7656925.0,663535.0,2017-02-20,2017-02-01
2,7200 BLOCK OF SE 32ND AVE,17-902450,PROPERTY,EASTMORELAND,2017-02-21,2017-02-01,2345,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.471859,-122.630327,7655675.0,665330.0,2017-02-22,2017-02-01
3,6500 BLOCK OF SE 32ND AVE,17-902495,PROPERTY,EASTMORELAND,2017-02-21,2017-02-01,2350,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.475196,-122.630444,7655677.0,666547.0,2017-02-22,2017-02-01
4,500 BLOCK OF N DIXON ST,17-901848,PROPERTY,ELIOT,2016-12-21,2016-12-01,1330,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.534551,-122.671730,7645672.0,688462.0,2017-02-11,2017-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202547,SE 3RD AVE / SE ALDER ST,17-904237,PROPERTY,BUCKMAN WEST,2017-03-25,2017-03-01,2330,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.517950,-122.662774,7647804.0,682350.0,2017-03-28,2017-03-01
202548,1000 BLOCK OF SE MORRISON ST,17-904282,PROPERTY,BUCKMAN WEST,2017-03-28,2017-03-01,200,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.517253,-122.655173,7649745.0,682044.0,2017-03-29,2017-03-01
202549,100 BLOCK OF SE STARK ST,17-904261,PROPERTY,BUCKMAN WEST,2017-03-29,2017-03-01,610,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.519425,-122.664286,7647431.0,682898.0,2017-03-29,2017-03-01
202550,1100 BLOCK OF SE 6TH AVE,17-904274,PROPERTY,BUCKMAN WEST,2017-03-28,2017-03-01,1800,LARCENY OFFENSES,1,THEFT FROM MOTOR VEHICLE,45.514730,-122.659690,7648563.0,681155.0,2017-03-29,2017-03-01


In [39]:
df.index = df.index.astype(int)

In [40]:
from sqlalchemy import create_engine

import pymysql

ModuleNotFoundError: No module named 'pymysql'

In [41]:
tableName = 'milestone2'
sqlEngine = create_engine('mysql+pymysql://root:@127.0.0.1/crime', pool_recycle=3600)
dbConnection = sqlEngine.connect()

ModuleNotFoundError: No module named 'pymysql'

In [42]:
try:
    frame = df.to_sql(tableName, dbConnection, if_exists='replace',index_label='id');
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%tableName);   
finally:
    dbConnection.close()

name 'dbConnection' is not defined


NameError: name 'dbConnection' is not defined